**Import the required modules**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

Data is manually downloaded from StatsBomb open source github repository. This is stored in the local project directory. The model we will be building is based on World Cup 2018 data.

In [2]:
#extracting information about which leagues
filepath = "C:\\Users\\anike\\PycharmProjects\\Soccer-Analytics\\Soccer-Analytics\\open-data\\data\\matches\\"
competitions =["43"]
league_list = []
for comp in competitions:
    path = filepath+comp
#     print(path)
    league_list.extend(glob.glob(path+"\\*.json"))

In [21]:
match_list

[7562,
 7549,
 7565,
 7581,
 7557,
 7566,
 7548,
 7534,
 7576,
 7540,
 7568,
 7569,
 7570,
 8655,
 8657,
 7543,
 7558,
 7529,
 8650,
 7546,
 7537,
 7580,
 7545,
 7536,
 7530,
 7535,
 7584,
 8649,
 7538,
 7533,
 7560,
 7525,
 7532,
 7555,
 7571,
 8652,
 7553,
 7559,
 7577,
 7539,
 7550,
 7556,
 7561,
 7586,
 7563,
 7579,
 7578,
 8651,
 7531,
 7552,
 8658,
 7585,
 7554,
 7542,
 7551,
 7564,
 7567,
 7544,
 7541,
 7583,
 7572,
 7547,
 7582,
 8656]

In [3]:
#extracting match specific information
match_list=[]
home_list=[]
match_dict={}
for league in league_list:
#     print(league)
    df_matches=pd.read_json(league)
    match_list.extend(df_matches['match_id'].values)
    for i in range(0,df_matches.shape[0]):
        home_list.append(df_matches['home_team'].values[i]['home_team_name'])

comb_list=list(zip(match_list,home_list))
match_dict={match:home for match,home in comb_list}

The data that contains shot specific information is extracted directly from the source.

In [4]:
eventspath = "C:\\Users\\anike\\PycharmProjects\\Soccer-Analytics\\Soccer-Analytics\\open-data\\data\\events\\"
shots_list=[] # all shots
loc_list=[] #all shot locations
timestamp_list=[] #time at which shot was taken
outcome_name_list = [] #all shot outcomes
outcome_id_list=[] #all shot outcome ids as per data
bodypart_name_list = [] #bodypart involved in shot
bodypart_id_list=[] #bodypart ids as per data
technique_name_list = [] #technique used in shot
technique_id_list=[] #technique ids as per data
type_name_list = [] #play pattern names
type_id_list=[] #play pattern ids
first_time_list=[] #whether shot was taken without a preceding touch 
statsbomb_xg_list = [] #statsbomb's xG prediction
key_pass_type_list=[] #the key pass leading up to the shot
key_pass_pattern_list=[] # type of key pass event
player_list=[] #the player who took the shot
home_or_away=[] #the team the player belongs to
preceding_event_list=[]#the event preceding the shot
preceding_event_player_list=[]#the player involved in preceding event
preceding_event_team_list=[]#the team of the player involved
cross_list=[]#whether a key pass is a cross
cutback_list=[]#whether a key pass is a cutback

In [5]:
#extracting the above information
for match in match_list:
    matchname = str(match)
    path = eventspath+matchname+".json"
    df_events=pd.read_json(path)
    df_shots = df_events[['index','timestamp','shot','location','player','possession_team']].dropna()
    shots_list.append(df_shots.dropna().shape[0])
    loc_list.extend(list(df_shots['location'].values))
    timestamp_list.extend(list(df_shots['timestamp'].values))
    for i in range(0,len(df_shots)):
        outcome_name_list.append(df_shots.iloc[i][2]['outcome']['name'])
        outcome_id_list.append(df_shots.iloc[i][2]['outcome']['id'])
        bodypart_name_list.append(df_shots.iloc[i][2]['body_part']['name'])
        bodypart_id_list.append(df_shots.iloc[i][2]['body_part']['id'])
        technique_name_list.append(df_shots.iloc[i][2]['technique']['name'])
        technique_id_list.append(df_shots.iloc[i][2]['technique']['id'])
        type_name_list.append(df_shots.iloc[i][2]['type']['name'])
        type_id_list.append(df_shots.iloc[i][2]['type']['id'])
        player_list.append(df_shots.iloc[i][4]['name'])
        if df_shots.iloc[i][5]['name'] == match_dict[match]:
            home_or_away.append('home')
        else:
            home_or_away.append('away')
#         team_list.append(df_shots.iloc[i][5]['name'])
        
        #Whether a shot was a first time shot is indicated by a Boolean. Many shots are not first time shots
        #Error handling is used to not stop data extraction
        try:
            first_time_list.append(df_shots.iloc[i][2]['first_time'])
        except:
            first_time_list.append(False)
        
        #Many shots do not have a preceding key pass. Error handling is used in such cases
        try:
            key_pass = df_shots.iloc[i][2]['key_pass_id']
            filt1 = df_events['id']==key_pass
            pass_info = df_events[filt1].dropna(axis='columns')['pass']
            key_pass_type_list.append(pass_info.iloc[0]['height']['name'])
            pattern_info = df_events[filt1].dropna(axis='columns')['play_pattern']
            key_pass_pattern_list.append(pattern_info.iloc[0]['name'])
            try:
                if pass_info.iloc[0]['cross']==True:
                    cross_list.append(True)
                else:
                    cross_list.append(False)
            except KeyError:
                cross_list.append(False)
            try:
                if pass_info.iloc[0]['cut_back']==True:
                    cutback_list.append(True)
                else:
                    cutback_list.append(False)
            except KeyError:
                cutback_list.append(False)
                
        except KeyError:
            key_pass_type_list.append(np.nan)
            cutback_list.append(False)
            cross_list.append(False)
            key_pass_pattern_list.append(np.nan)
        #Extracting the event preceding the shot
        idx_filt = df_shots.iloc[i,0]-1
        filt = df_events['index']==idx_filt
        df_events_filt=df_events[filt].dropna(axis='columns')
        preceding_event_list.append(df_events_filt['type'].values[0]['name'])
        try:
            preceding_event_player_list.append(df_events_filt['player'].values[0]['name'])
        except:
            preceding_event_player_list.append(np.nan)
        preceding_event_team_list.append(df_events_filt['possession_team'].values[0]['name'])
        
        #extracting statsbomb xG
        statsbomb_xg_list.append(df_shots.iloc[i][2]['statsbomb_xg'])


The most interesting things about the data is that it contains the exact locations of all players at the time of the shot. The data has to be worked upon in a way so that is can be useful for us.

The following function calculates the number of players between the shot and the goal. It makes use of a baycentric technique(link - http://blackpawn.com/texts/pointinpoly/), where in it considers the triangle created by shot location and edges of goal. Further, it evaluates each player location as to whether it lies in the triangle. Number of players in the triangle will be stored as the 'pack_density'.

In [6]:
def vectorization(pointa, pointb):
    '''
    Given two points, create a vector from the first to the second
    Params:
        pointa, pointb - x,y co-ordinates (list)
    Returns:
        vector from point a to point b
    '''
    return [pointb[0]-pointa[0], pointb[1]-pointa[1]]


def liesInside(point_shot, point_goal1, point_goal2, point_freeze):
    '''
    Given the 3 vertices of a triangle, this function determines if a 4th
    point lies inside the triangle
    Params:
        point_shot - shot location x,y (list)
        point_goal1 and point_goal2 - goal edges x,y (list)
        point_freeze - player location x,y(list)
    Return:
        True or False (boolean)
    
    '''
    v2 = vectorization(point_freeze, point_shot)
    v0 = vectorization(point_goal1, point_shot)
    v1 = vectorization(point_goal2, point_shot)

    u = (np.dot(v1, v1)*np.dot(v2, v0)-np.dot(v1, v0)*np.dot(v2, v1)) / \
        (np.dot(v0, v0)*np.dot(v1, v1)-np.dot(v0, v1)*np.dot(v1, v0))
    v = (np.dot(v0, v0)*np.dot(v2, v1)-np.dot(v0, v1)*np.dot(v2, v0)) / \
        (np.dot(v0, v0)*np.dot(v1, v1)-np.dot(v0, v1)*np.dot(v1, v0))

    if u < 0 or v < 0:
        return False
    elif u > 1 or v > 1:
        return False
    elif u+v > 1:
        return False
    else:
        return True
    
#defining edges of goal
goal_x1y1 = [120,36]
goal_x2y2 = [120,44]
pack_density_list = []

In [7]:
for match in match_list:
    matchname = str(match)
    path = eventspath+matchname+".json"
    df_events=pd.read_json(path)
    df_shots = df_events[['index','shot','location','player','possession_team']].dropna()
    
    for i in range(0,len(df_shots)):
        shot_loc=df_shots['location'].iloc[i]
        freeze_frame_list=[]
        # in cases of penalties, a freeze frame is not provided so a KeyError will be raised
        # using error handling to append 1 in such cases
        try:
            for j in range(0,len(df_shots['shot'].iloc[i]['freeze_frame'])):
                location=df_shots['shot'].iloc[i]['freeze_frame'][j]['location']
                freeze_frame_list.append(liesInside(shot_loc,goal_x1y1,goal_x2y2,location))
        except KeyError:
            freeze_frame_list.append(1) 
        pack_density_list.append(sum(freeze_frame_list))

Euclidean distance is used to calculate the distance of shot location from center of the goal and and angle made between shot location and the goal edges. 
Angle is calculated using the Cosine rule.

In [8]:
df = pd.DataFrame(loc_list)

In [9]:
df.columns = ['shot_x','shot_y']
df['shot_distance']=round(np.sqrt((df['shot_x']-120)**2+(df['shot_y']-40)**2),1)
df['shot_a'] = np.sqrt((df['shot_x']-120)**2+(df['shot_y']-36)**2)
df['shot_b']=np.sqrt((df['shot_x']-120)**2+(df['shot_y']-44)**2)
df['shot_angle']=np.arccos((df['shot_a']**2+df['shot_b']**2-64)/(2*df['shot_a']*df['shot_b']))#calculating angle using cosine rule
df['shot_angle']=np.degrees(df['shot_angle'])
df['shot_angle']=round(df['shot_angle'],1)

Adding the remaining features to the dataframe.

In [10]:
df['timestamp']=timestamp_list
df['outcome_name']=outcome_name_list
df['outcome_id']=outcome_id_list
df['body_part_name']=bodypart_name_list
df['body_part_id']=bodypart_id_list
df['technique_name']=technique_name_list
df['technique_id']=technique_id_list
df['type_name']=type_name_list
df['type_id']=type_id_list
df['first_time']=first_time_list
df['preceding_event']=preceding_event_list
df['pack_density']=pack_density_list
df['shot_player']=player_list
df['home_or_away']=home_or_away
df['key_pass_type']=key_pass_type_list
df['key_pass_pattern']=key_pass_pattern_list
df['isCross']=cross_list
df['isCutback']=cutback_list
df['preceding_event']=preceding_event_list
df['preceding_event_player']=preceding_event_player_list
df['preceding_event_team']=preceding_event_team_list
df['statsbomb_xg']=statsbomb_xg_list

In [11]:
df.isna().sum()

shot_x                      0
shot_y                      0
shot_distance               0
shot_a                      0
shot_b                      0
shot_angle                  0
timestamp                   0
outcome_name                0
outcome_id                  0
body_part_name              0
body_part_id                0
technique_name              0
technique_id                0
type_name                   0
type_id                     0
first_time                  0
preceding_event             0
pack_density                0
shot_player                 0
home_or_away                0
key_pass_type             526
key_pass_pattern          526
isCross                     0
isCutback                   0
preceding_event_player     13
preceding_event_team        0
statsbomb_xg                0
dtype: int64

There are some missing values. A lot of shots don't have an associated key pass which is expected as many shots don't have a key pass before it for e.g a direct freekick. These missing values will be handled later. 13 preceding_event_player values are missing. Let us see why...


In [13]:
#analyzing missing shot_angle
filt1=df['preceding_event_player'].isna()
df[filt1][['preceding_event','preceding_event_player','preceding_event_team']]

,preceding_event,preceding_event_player,preceding_event_team
36,Tactical Shift,NaN,Nigeria
77,Camera On,NaN,Denmark
108,Half Start,NaN,Denmark
268,Camera On,NaN,Russia
430,Injury Stoppage,NaN,Spain
926,Camera On,NaN,Croatia
928,Half Start,NaN,Croatia
994,Tactical Shift,NaN,Saudi Arabia
1138,Camera On,NaN,Switzerland
1367,Half Start,NaN,England


We can see that the preceding events cannot be linked to a player. And the corresponding events were not that important in context of shots. Thus, it makes sense to drop these rows. 

In [14]:
df=df.dropna(how='any',subset=['preceding_event_player'])

In [15]:
df['key_pass_pattern'].value_counts()

Regular Play      520
From Throw In     201
From Corner       200
From Free Kick    161
From Counter       44
From Goal Kick     30
From Keeper        14
From Kick Off       7
Other               2
Name: key_pass_pattern, dtype: int64

Since a lot of shots don't have an associated key pass prior to the shot. We assign the value 'None' to shots which are missing key pass information.

In [16]:
key_pass_type_list=[]
key_pass_pattern_list=[]
for index,row in df.iterrows():
        if row['key_pass_type'] not in ['Ground Pass','Low Pass','High Pass']:
            key_pass_type_list.append('None')
            key_pass_pattern_list.append('None')
        else:
            key_pass_type_list.append(row['key_pass_type'])
            key_pass_pattern_list.append(row['key_pass_pattern'])

In [17]:
df['key_pass_type']=key_pass_type_list
df['key_pass_pattern']=key_pass_pattern_list

In [18]:
df.isna().sum()

shot_x                    0
shot_y                    0
shot_distance             0
shot_a                    0
shot_b                    0
shot_angle                0
timestamp                 0
outcome_name              0
outcome_id                0
body_part_name            0
body_part_id              0
technique_name            0
technique_id              0
type_name                 0
type_id                   0
first_time                0
preceding_event           0
pack_density              0
shot_player               0
home_or_away              0
key_pass_type             0
key_pass_pattern          0
isCross                   0
isCutback                 0
preceding_event_player    0
preceding_event_team      0
statsbomb_xg              0
dtype: int64

Now that there are no missing values in the dataframe, we go ahead and pickle(a serialized way of storing a Pandas dataframe) this dataframe.

In [19]:
df.to_pickle('./WC2018_preprocessed.pkl'